# Amazon SageMaker Model Bias Monitors: Fairness and Explainability with SageMaker Clarify

O Amazon SageMaker Clarify ajuda a melhorar os modelos de machine learning ao detectar possíveis vieses e ajudar a explicar como esses modelos fazem previsões. A funcionalidade de equidade e explicabilidade fornecida pelo SageMaker Clarify dá um passo para permitir que os clientes da AWS criem modelos de machine learning confiáveis e compreensíveis. O produto vem com as ferramentas para ajudar você nas tarefas a seguir.

- Avaliar os vieses que podem ocorrer durante cada estágio do ciclo de vida da ML (coleta de dados, treinamento e ajuste do modelo, além de monitoramento dos modelos de ML implantados para inferência).
- Gerar relatórios de governança de modelo direcionados a equipes de risco e conformidade e reguladores externos.
- Fornecer explicações sobre os dados, os modelos e o monitoramento usados para avaliar as previsões.

## Importações

In [ ]:
import os
import copy
import json
import random
import time
import datetime
import boto3
import pandas as pd

from datetime import datetime, timedelta

from sagemaker import get_execution_role, image_uris, Session
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.predictor import Predictor

from sagemaker.model_monitor import (
    BiasAnalysisConfig,
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
    ExplainabilityAnalysisConfig,
    ModelBiasMonitor,
    ModelExplainabilityMonitor,
)

from sagemaker.clarify import (
    BiasConfig,
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)

### Configuração para esta tarefa

In [ ]:
# Substitua <bucket_name> abaixo pelo valor modelArtifactBucket à esquerda da página de instrução
bucket = '<bucket_name>'

# Obter o nome do endpoint de produção
endpoint_name = boto3.Session().client('sagemaker').list_endpoints(SortBy='CreationTime')['Endpoints'][0]['EndpointName']

# Obter o nome do modelo de produção
model_name = boto3.Session().client('sagemaker').list_models(SortBy='CreationTime')['Models'][0]['ModelName']


print(f"Bucket Name :{bucket}")
print(f"Production Endpoint Name:{endpoint_name}")
print(f"Production Model Name:{model_name}")

In [ ]:
role = get_execution_role()
print(f"RoleArn: {role}")

sagemaker_session = Session()
sagemaker_client = sagemaker_session.sagemaker_client
sagemaker_runtime_client = sagemaker_session.sagemaker_runtime_client

region = sagemaker_session.boto_region_name
print(f"AWS region: {region}")

In [ ]:
print(f"Demo Bucket: {bucket}")
prefix = f'sagemaker/DEMO-ClarifyModelMonitor-{datetime.now():%Y%m%d%H%M%S}'
s3_key = f"s3://{bucket}/{prefix}"
print(f"S3 key: {s3_key}")

code_prefix = '{}/code'.format(prefix)
data_capture_prefix = f'{prefix}/datacapture'
s3_capture_upload_path = f'{s3_key}/datacapture'
s3_report_path = f'{s3_key}/reports'

print(f"Capture path: {s3_capture_upload_path}")
print(f"Report path: {s3_report_path}")

endpoint_instance_count = 1
endpoint_instance_type = "ml.m5.xlarge"

### Arquivos de dados

Leia os arquivos de dados

In [ ]:
test_dataset = "data/test.csv"
validation_dataset = "data/train.csv"
dataset_type = "text/csv"

with open(validation_dataset) as f:
    headers_line = f.readline().rstrip()
all_headers = headers_line.split(",")
label_header = all_headers[0]

### Ativar a captura de dados no endpoint

Para ativar a captura de dados para as tarefas de monitoramento, especifique aqui a nova opção de captura chamada DataCaptureConfig, que permite capturar o payload de solicitação e o payload de resposta do endpoint. A configuração de captura aplica-se a todas as variantes. 

Esta operação leva geralmente de 10 a 15 minutos.

**Observação:** isso pode ser automatizado via cloudformation

In [ ]:
data_capture_config = DataCaptureConfig(
                        enable_capture = True,
                        sampling_percentage=100,
                        destination_s3_uri=s3_capture_upload_path,
                        kms_key_id=None,
                        capture_options=["REQUEST", "RESPONSE"],
                        csv_content_types=["text/csv"],
                        json_content_types=["application/json"])



# Agora é hora de aplicar a nova configuração e esperar que ela seja aplicada
predictor = Predictor(endpoint_name=endpoint_name)
predictor.update_data_capture_config(data_capture_config=data_capture_config)
sagemaker_session.wait_for_endpoint(endpoint=endpoint_name)

### Testar o endpoint com o tráfego de amostra

In [ ]:
# Aqui estamos escolhendo aleatoriamente um subconjunto de dados de conjuntos de dados de teste.
import itertools

shape = pd.read_csv(test_dataset)

a = [10*i for i in range(3)]
b = [10+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data = shape.drop(shape.columns[[0]],axis=1)
test_data = test_data.iloc[indices]
test_data_with_label = shape.iloc[indices]

In [ ]:
test_data.to_csv("data/data-test.csv",index=False,header=False)
test_data_with_label.to_csv("data/data-test-label.csv",index=False,header=False)

In [ ]:
# Invocar o endpoint com dados de teste para um teste rápido
print(f"Sending test traffic to the endpoint {endpoint_name}. \nPlease wait...")
predictions = ''

with open('data/data-test.csv', 'r') as f:
    for row in f:
        print(".", end="", flush=True)
        payload = row.rstrip('\n')
        response = sagemaker_runtime_client.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType="text/csv",
                                   Body=payload)
        predictions = ','.join([predictions, response['Body'].read().decode('utf-8')])
        time.sleep(0.5)

predictions = predictions.replace('\n','')
predictions = predictions.split(",")
predictions.pop(0)
print("="*20)
print(predictions)
print("Done!") 

## Visualizar dados capturados

Agora liste os arquivos de captura de dados armazenados no Amazon S3. É provável que você veja arquivos diferentes em diferentes períodos de tempo organizados com base na hora em que a invocação ocorreu. O formato do caminho do Amazon S3 é:

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [ ]:
s3_client = boto3.Session().client('s3')
current_endpoint_capture_prefix = '{}/{}'.format(data_capture_prefix, endpoint_name)
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
print(result)
str_error = "waiting for the file to load..."
while str_error:
    try:
        capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
        print(capture_files)
        str_error=None
    except:
        print(str_error)
        time.sleep(10)
        contine
print("Found Capture Files:")
print("\n ".join(capture_files))

Em seguida, visualize o conteúdo de um único arquivo de captura. Aqui você deve ver todos os dados capturados em um arquivo formatado JSON específico do Amazon SageMaker. Dê uma olhada rápida nas primeiras linhas do arquivo capturado.

In [ ]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

Por fim, o conteúdo de uma única linha está presente abaixo em um arquivo JSON formatado para que você possa observar um pouco melhor.

In [ ]:
import json
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

### Configuração do Amazon Sagemaker Clarify

**Observação:** as amostras de código a seguir são retiradas do [código de exemplo] (https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker_processing/fairness_and_explainability/fairness_and_explainability.ipynb) fornecido pelo Amazon sagemaker. Alguns dos códigos são atualizados de acordo com os requisitos do laboratório.

In [ ]:
# Executar o processador do SageMaker Clarify usando o comando a seguir
from sagemaker import clarify
clarify_processor = clarify.SageMakerClarifyProcessor(role=role,
                                                      instance_count=1,
                                                      instance_type='ml.m5.xlarge',
                                                      sagemaker_session=sagemaker_session)

Faça upload dos conjuntos de dados de teste e treinamento para o s3. O _train_uri_ e o _test_uri_ serão usados para extrair dados diretamente da configuração de dados e da configuração de modelos de gravação do S3 nas próximas etapas.

In [ ]:
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

train_uri = S3Uploader.upload('data/train.csv', 's3://{}/{}'.format(bucket, prefix))
train_input = TrainingInput(train_uri, content_type='csv')
test_uri = S3Uploader.upload('data/test.csv', 's3://{}/{}'.format(bucket, prefix))

### Detectar vieses

O SageMaker Clarify ajuda a detectar possíveis vieses de pré e pós-treinamento usando uma variedade de métricas.

### Gravar o DataConfig e o ModelConfig

Um objeto **DataConfig** comunica algumas informações básicas sobre a E/S de dados para o SageMaker Clarify. Você especificará onde encontrar o conjunto de dados de entrada, onde armazenar a saída, a coluna de destino (rótulo), os nomes dos cabeçalhos e o tipo de conjunto de dados.

In [ ]:
bias_report_output_path = 's3://{}/{}/clarify-bias'.format(bucket, prefix)
bias_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                      s3_output_path=bias_report_output_path,
                                      label='Churn',
                                      headers=all_headers,
                                      dataset_type='text/csv')

Um objeto **ModelConfig** comunica informações sobre seu modelo treinado. Para evitar tráfego adicional para seus modelos de produção, o SageMaker Clarify configura e desativa um endpoint dedicado durante o processamento.

- **instance_type** e **instance_count** especificam o tipo de instância e a contagem de instância preferidos usados para executar seu modelo durante o processamento do SageMaker Clarify. O conjunto de dados de teste é pequeno, portanto, uma única instância básica é boa o suficiente para executar este exemplo. Se você tiver um grande conjunto de dados complexo, talvez possa usar um tipo de instância mais adequado para acelerar ou adicionar mais instâncias para ativar a paralelização do Spark.

- **accept_type** indica o formato do payload de resposta do endpoint e **content_type** indica o formato do payload de solicitação para o endpoint.

In [ ]:
model_config = clarify.ModelConfig(model_name=model_name,
                                   instance_type='ml.m5.xlarge',
                                   instance_count=1,
                                   accept_type='text/csv',
                                   content_type='text/csv')

Um **ModelPredictedLabelConfig** fornece informações sobre o formato de suas previsões. O modelo Floresta Aleatória gera probabilidades de amostras, de modo que o SageMaker Clarify invoca o endpoint e, em seguida, usa o probability_threshold para converter a probabilidade em rótulos binários para análise de tendência. A previsão acima do limite é interpretada como o valor de rótulo 1 e abaixo ou igual ao valor de rótulo 0.

In [ ]:
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

### Gravar o BiasConfig
O SageMaker Clarify também precisa de informações sobre quais são as colunas sigilosas (facets), quais podem ser os recursos confidenciais (facet_values_or_threshold) e quais são os resultados desejáveis (label_values_or_threshold). O SageMaker Clarify pode lidar com os dados categóricos e contínuos para facet_values_or_threshold e para label_values_or_threshold. Nesse caso, estamos usando os dados categóricos.

Especificamos essas informações na API BiasConfig. Aqui, em que o resultado positivo é Churn = 0, _Account Length_ é uma categoria sigilosa e os respondentes _Day Calls_ são o grupo sensível. group_name é usado para formar subgrupos para a medição.

In [ ]:
bias_config = clarify.BiasConfig(label_values_or_threshold=[0],
                                facet_name='Account Length',
                                facet_values_or_threshold=[100],
                                group_name='Day Calls')

**Viés de pré-treinamento**

O viés pode estar presente nos dados antes que ocorra qualquer treinamento do modelo. Inspecionar os dados em busca de vieses antes do início do treinamento pode ajudar a detectar lacunas de coleta de dados, informar a engenharia de recursos, além de ajudar a entender quais vieses sociais os dados podem refletir.

O cálculo das métricas de viés de pré-treinamento não requer um modelo treinado.

**Viés de pós-treinamento**

O cálculo das métricas de viés de pós-treinamento requer um modelo treinado.

Os dados de treinamento não tendenciosos (conforme determinado pelos conceitos de equidade medidos pela métrica de viés) ainda podem resultar em previsões de modelo tendencioso após o treinamento. Essa possibilidade depende de vários fatores, incluindo escolhas de hiperparâmetros.

Você pode executar essas opções separadamente com run_pre_training_bias() e run_post_training_bias() ou ao mesmo tempo com run_bias(), conforme mostrado a seguir.

In [ ]:
clarify_processor.run_bias(data_config=bias_data_config,
                           bias_config=bias_config,
                           model_config=model_config,
                           model_predicted_label_config=predictions_config,
                           pre_training_methods='all',
                           post_training_methods='all')

**Visualizar o relatório de viés**

No Studio, você pode visualizar os resultados na guia de experimentos.

<img src="./recordings/bias_report.gif">

Cada métrica de viés tem explicações detalhadas com exemplos que você pode explorar.

<img src="./recordings/bias_detail.gif">

Os resultados também podem ser resumidos em uma tabela prática.

<img src="./recordings/bias_report_chart.gif">



Se você ainda não é um usuário do Studio, acesse o relatório de viés nos formatos pdf, html e ipynb no bucket do S3 a seguir:

In [ ]:
bias_report_output_path

### Explicando previsões

Há necessidades de negócios em expansão e regulamentações legislativas que exigem explicações sobre o motivo de um modelo ter tomado a decisão que tomou. O SageMaker Clarify usa o SHAP para explicar a contribuição de cada recurso de entrada para a decisão final.

O algoritmo Kernel SHAP requer uma linha de base (também conhecida como conjunto de dados do histórico). O tipo de conjunto de dados da linha de base deve ser o mesmo que dataset_type de DataConfig e as amostras de linha de base devem incluir apenas recursos. Por definição, a linha de base deve ser um URI do S3 para o arquivo de conjunto de dados da linha de base ou uma lista local de amostras. Nesse caso, escolhemos o último e colocamos a primeira amostra do conjunto de dados de teste na lista.

In [ ]:
shap_config = clarify.SHAPConfig(baseline=[test_data.iloc[0].values.tolist()],
                                 num_samples=15,
                                 agg_method='mean_abs',
                                 save_local_shap_values=False)

explainability_output_path = 's3://{}/{}/clarify-explainability'.format(bucket, prefix)
explainability_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                s3_output_path=explainability_output_path,
                                label='Churn',
                                headers=all_headers,
                                dataset_type='text/csv')

In [ ]:
clarify_processor.run_explainability(data_config=explainability_data_config,
                                     model_config=model_config,
                                     explainability_config=shap_config)

#### Visualizar o relatório de explicabilidade
Assim como no relatório de viés, você pode visualizar o relatório de explicabilidade no Studio na guia de experimentos


<img src="./recordings/explainability_detail.gif">

A guia Informações do modelo contém links diretos para o relatório e as informações do modelo.

Se você ainda não é um usuário do Studio, como no relatório de viés, pode acessá-lo no bucket do S3 a seguir:

In [ ]:
explainability_output_path

### Limpeza

Finalmente, não se esqueça de limpar os recursos que configuramos e usamos para esta demonstração.

In [ ]:
session.delete_model(model_name)